# Graph Neural Network

## Introducing the Model

We have already laid out the basic idea of Graph Neural Networks (GNNs) in the introduction, and as such we will instead introduce a useful variant, known as Graph Convolutional Networks (GCNs). GCNs are the most prevelant and broadly applied model of GNNs, [1] which are able to leverage both the information of the nodes and their locality to make predictions [2]. They do this using a convolution layer, which merges the features of a node with those of its neighbours. The most common method of recommender systems used with GCNs is Collaborative Filtering, where user's past interactions and connections to other users is used to make predictions. The main way it does this is through learning latent features (or an embedding) for each user and item node. GCNs then update this embedding by propagating information across the graph:

Let $\mathbf{e}_u^{(0)}$ denote the user embedding of user $u$, and $\mathbf{e}_i^{(0)}$ the item embedding of item $i$. Then after $k$ layers of propagation, we have:

$$\mathbf{e}_u^{(k+1)} = \sigma \left( \mathbf{W}_1 \mathbf{e}_u^{(k)} + \sum_{i\in \mathcal{N}_u} \frac{1}{\sqrt{|\mathcal{N}_u||\mathcal{N}_i|}} (\mathbf{W}_1\mathbf{e}_u^{(k)} + \mathbf{W}_2(\mathbf{e}_i^{(k)}\odot \mathbf{e}_u^{(k)}))\right)$$

and a similar equation for $\mathbf{e}_i^{(k+1)}$ [3]. In this equation, $\sigma$ is the nonlinear activation function (e.g. ReLU), $\mathcal{N}_u$ denotes the set of items that are interacted with by user $u$, and similar for $\mathcal{N}_i$ and item $i$. $\mathbf{W}_1$ and $\mathbf{W}_2$ are trainable weight matrices used to perform feature transformations in each layer.

These equations represent the basic building block of most GCNs, however after recent investigations as published in paper [3], it was discovered that for recommendation systems in particular, both the activation function, $\sigma$, and the feature transformation matrices, $\mathbf{W}_1, \mathbf{W}_2$, have little influence on the model and in fact make the predictions worse in some situations. As such a new model was suggested that stripped these parts for a much simpler propogation method:

$$\mathbf{e}_u^{(k+1)} = \sum_{i\in \mathcal{N}_u} \frac{1}{\sqrt{|\mathcal{N}_u||\mathcal{N}_i|}} \mathbf{e}_i^{(k)}$$

$$\mathbf{e}_i^{(k+1)} = \sum_{u\in \mathcal{N}_i} \frac{1}{\sqrt{|\mathcal{N}_i||\mathcal{N}_u|}} \mathbf{e}_u^{(k)}$$

This model is known as LightGCN, and was shown to perform better in most cases than the other prevalent GCN and GNN models for recommendation systems. This goes to show that more complicated is not always better, which carries over to data at scale. Sometimes one of the best ways to deal with big data is to make sure your model isn't performing unnecessary calculations.

LightGCN is still among the best models for collaborative filtering in recommender systems, and as such is the one that we will consider here.

## Importing Modules and Reading Data

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from IPython import get_ipython

sys.path.append(os.path.join('..','scripts'))

import GraphFuncs as gf

In [ ]:
artists = pd.read_csv(os.path.join('..','data','artists.dat'), delimiter='\t')
tags = pd.read_csv(os.path.join('..','data','tags.dat'), delimiter='\t',encoding='ISO-8859-1')
user_artists = pd.read_csv(os.path.join('..','data','user_artists.dat'), delimiter='\t')
user_friends = pd.read_csv(os.path.join('..','data','user_friends.dat'), delimiter='\t')
user_taggedartists_timestamps = pd.read_csv(os.path.join('..','data','user_taggedartists-timestamps.dat'), delimiter='\t')
user_taggedartists = pd.read_csv(os.path.join('..','data','user_taggedartists.dat'), delimiter='\t')

## Deciding the Question

The goal of a recommender system is to try to predict which items a user will like, and then give them recommendations based on this. In this section, for our dataset, we will look at the goal of predicting whether or not a user will like an artist. To achieve this, we will create an 80-20 training-testing split of the data. We have seen that for most users, the `user_artist` dataset contains their top 50 artists, so we will take 40 of them for each user at random, and isolate the other 10 to use as testing data. We will then train a GNN on the 40 artists in the training dataset, and evaluate the model by seeing how it predicts if the user will like the 10 artists in the testing data. If the model performs well on this target, then we could use the model to predict whether or not a user will like an artist not contained in their 50 artists.

## Creating the Train-Test Split

In order to create the train-test split we will use the Scikit-learn module, which has a built in function for it. It also has a built in `stratify` parameter, which allows us to specify that the data should be split whilst keeping the proportion of data the same for each user, i.e. what we described above. However this requires that each user has atleast 2 artist data entries, but we see that this is not the case:

In [ ]:
users = user_artists['userID'].unique()
singleartistusers = [int(user) for user in users if len(gf.get_artists(user,user_artists)) == 1]
print(singleartistusers)

We see that there are 8 users with only 1 artist data entry. As such we remove those, create the train-test split, and then re-add those users back to the training data:

In [ ]:
# Isolate the rows of data for the users which only have one artist data point.
singleartistusersdf = user_artists[user_artists['userID'].isin(singleartistusers)]
# Take the rest of the data into a temp df
user_artists_temp = user_artists[~user_artists['userID'].isin(singleartistusers)]

# Create the train-test split using this temp df
user_artists_train, user_artists_test = train_test_split(user_artists_temp,
                                                          test_size = 0.2, 
                                                          stratify = user_artists_temp['userID'], 
                                                          random_state = 47)

# Concat the dfs back together
user_artists_train = pd.concat([user_artists_train,singleartistusersdf])

Then we save the datasets as text files so that the model can access them.

In [ ]:
dfs = [user_artists_train, user_artists_test, user_friends]

filepath = os.path.join('..','data','GNN')
if not os.path.exists(filepath):
    os.makedirs(filepath)

for df in dfs:
    df.to_csv(os.path.join(filepath,gf.get_df_name(df, globals()) + '.txt'),sep='\t',header=False,index=False)

## Exploring the Model

In order to use the LightGCN model, we have downloaded the scripts from the GitHub repository [4] associated with the paper [3] that suggested the model. This repository already had a Last.fm dataset built in, however it is different to ours and as such we must alter the code to run to our needs. This mostly involved adding a new class in the `dataloader.py` script, known as LastFM2, which was done by altering the already existing LastFM class.

In [ ]:
with open(os.path.join('..','scripts','LightGCN','dataloader.py')) as file:
    content = file.read()
    start = content.index('class LastFM2')
    print(content[start:start+1000] + ' \nOutput Truncated.')

We have also made other alterations to the code to suit our needs for the model. These changes will be annotated in the scripts themselves with comments starting with 'SH8', and mentioned in this document where appropriate.

The downloaded scripts are contained in the `../scripts/LightGCN` directory.

We have also made other alterations to the code to suit our needs for the model. These changes will be annotated in the scripts themself where appropriate.

## Testing the Model

To run the model, we use the `%run` magic command to access and run the script. Here we will run a small test of 10 epochs just to check that the model is working correctly. An epoch is a single run of the data through the algorithm. The more epochs we run, the better trained the model will be.

In [ ]:
# This cell is creating the results path so that performance data can be saved from the model. This is not used here but the folder needs to exist so that the model can run.
resultspath = os.path.join('..','scripts','LightGCN','results')
if not os.path.exists(resultspath):
    os.makedirs(resultspath)
resultsfile = os.path.join(resultspath,'results.csv')
if os.path.exists(resultsfile):
    os.remove(resultsfile)
pd.DataFrame(columns=['precision','recall','ndcg']).to_csv(resultsfile)

In [ ]:
%run -i ../scripts/LightGCN/main.py --decay=1e-4 --lr=0.001 --layer=3 --seed=47 --dataset="lastfm2" --topks="[20]" --recdim=64 --epochs=10 --device=CPU

We see that even just for 10 epochs, the model takes a very long time to train (around 70 seconds in our tests). This is because we are running it on the device's CPU. When it comes to machine learning, and especially for big datasets, running the model on the CPU is usually very slow and not ideal. Instead a GPU is often used since GPUs are designed to perform many parallel operations, which is very useful for training neural network models on large datasets. PyTorch and other frameworks for neural networks have this feature built in, and with the help of the CUDA platform we can efficiently use a GPU to run the models if one is available. CUDA is a platform published by NVIDIA which allows for general-purpose processing on their GPUs [5,6]. This section will use CUDA 12.4 to run the model on a NVIDIA RTX 3060 Ti GPU:

In [ ]:
%run -i ../scripts/LightGCN/main.py --decay=1e-4 --lr=0.001 --layer=3 --seed=47 --dataset="lastfm2" --topks="[20]" --recdim=64 --epochs=10 --device=GPU

In our tests, we have seen that on a GPU, the time taken has gone from around 70 seconds to just 15 seconds. A significant time improvement, especially if we want to run the models for more epochs, which we often will. For this model, we often want to run the model for hundreds or possibly even a thousand epochs.

## Tuning the Model

LightGCN has a few parameters that we may want to tune. This includes but is not limited to:

* The number of layers: `--layer`. In a GCN the number of layers represents how far we want information to propagate across the graph. 1 layer means that a node will only receive information from its direct neighbours, whilst 2 layers would provide information from a node's neighbour's neighbours as well, and so on.
* The embedding size of the nodes: `--recdim`. This represents the number of features each node embedding has, essentially dictating the capacity of information that each node can hold.
* The learning rate: `--lr`, i.e. how much the model is adjusted with respect to the loss function during training. A high learning rate might make the model converge quickly to a solution, but this solution may not be optimal. A lower learning rate means the model might take a long time or get stuck in a local minima.

To better investigate tuning these parameters, we write a function to allow us to easily run the model whilst changing parameters, e.g. in a for loop. We do this by using the IPython module, which lets you run line magic from a string, which we can thus format:

In [ ]:
def LightGCN(decay = 1e-4, lr = 0.001, layer = 3, seed = 47, dataset = "lastfm2", topks = 20, recdim = 64, epochs = 100, device = 'GPU'):
    resultspath = os.path.join('..','scripts','LightGCN','results')
    if not os.path.exists(resultspath):
        os.makedirs(resultspath)
    filename = f'results_l{layer}_rd{recdim}_lr{lr}.csv'
    resultsfile = os.path.join(resultspath,filename)
    if os.path.exists(resultsfile):
        os.remove(resultsfile)
    pd.DataFrame(columns=['precision','recall','ndcg']).to_csv(resultsfile)
    get_ipython().run_line_magic('run', f'-i ../scripts/LightGCN/main.py --decay={decay} --lr={lr} --layer={layer} --seed={seed} --dataset={dataset} --topks="[{topks}]" --recdim={recdim} --epochs={epochs} --device={device}')

Since our goal is to train a model that works well to generate new predictions, we will use the testing data as validation data to evaluate the performance of the model, and then use the model to predict if a user will like artists that we do not have data on them liking.

The model will be evaluated every 10 epochs on three metrics:

* precision at k: This represents how many out of the top k predictions are present in the testing dataset.
* recall at k: This represents how many of the items in the testing dataset are present in the top k predictions.
* ndcg at k: NDCG stands for Normalized Discounted Cumulative Gain, and is a metric that compares predicted rankings of recommendations to an ideal order where all the testing datapoints are at the top of the list.

These may also be written as `precision@k`, `recall@k`, and `ndcg@k`, where this k is the same as the `--topks` parameter of the model.

In [ ]:
# for layers in [1,2,3]:
#     for recdims in [16,32,48,64]:
#         for lrs in [0.1,0.01,0.001]:
#             LightGCN(layer = layers, recdim = recdims, lr = lrs, epochs=500)

In [ ]:
# results = os.listdir('../scripts/LightGCN/results')
# for result in results:
#     resultdf = pd.read_csv(os.path.join('..','scripts','LightGCN','results',result),index_col='Unnamed: 0')
#     plt.figure()
#     plt.plot(resultdf['precision'])
#     plt.plot(resultdf['recall'])
#     plt.plot(resultdf['ndcg'])
#     plt.title(result)

# References

[1] Papers With Code: Graph Models - https://paperswithcode.com/methods/category/graph-models

[2] Towards Data Science: Graph Convolutional Networks - https://towardsdatascience.com/graph-convolutional-networks-introduction-to-gnns-24b3f60d6c95

[3] Xiangnan He, et al. 2020. LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation - https://arxiv.org/abs/2002.02126

[4] Guyse1234 Github Repository: LightGCN-PyTorch - https://github.com/gusye1234/LightGCN-PyTorch

[5] NVIDIA: CUDA Toolkit - https://developer.nvidia.com/cuda-toolkit

[6] Towards Data Science: Why Deep Learning Models Run Faster on GPUs: A Brief Introduction to CUDA Programming - https://towardsdatascience.com/why-deep-learning-models-run-faster-on-gpus-a-brief-introduction-to-cuda-programming-035272906d66